In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [3]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 80

In [4]:
train_dir = '/content/drive/MyDrive/VIPERC/dataset/train'
test_dir = '/content/drive/MyDrive/VIPERC/dataset/test'

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 5025 images belonging to 13 classes.
Found 1256 images belonging to 13 classes.


In [5]:
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

for layer in base_model.layers:
    layer.trainable = False

29084464/29084464 [==============================] - 0s 0us/step


In [6]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [7]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min')

In [9]:
model.fit(train_generator, epochs=EPOCHS, validation_data=test_generator, callbacks=[checkpoint, early_stopping])

Epoch 1/80
158/158 [==============================] - 46s 218ms/step - loss: 0.3288 - accuracy: 0.9065 - val_loss: 0.0589 - val_accuracy: 0.9817
Epoch 2/80
158/158 [==============================] - 29s 180ms/step - loss: 0.0545 - accuracy: 0.9847 - val_loss: 0.0426 - val_accuracy: 0.9889
Epoch 3/80
158/158 [==============================] - 27s 169ms/step - loss: 0.0213 - accuracy: 0.9952 - val_loss: 0.0525 - val_accuracy: 0.9833
Epoch 4/80
158/158 [==============================] - 27s 171ms/step - loss: 0.0111 - accuracy: 0.9972 - val_loss: 0.0442 - val_accuracy: 0.9881
Epoch 5/80
158/158 [==============================] - 28s 179ms/step - loss: 0.0222 - accuracy: 0.9932 - val_loss: 0.0234 - val_accuracy: 0.9928
Epoch 6/80
158/158 [==============================] - 28s 177ms/step - loss: 0.0077 - accuracy: 0.9976 - val_loss: 0.0220 - val_accuracy: 0.9936
Epoch 7/80
158/158 [==============================] - 27s 173ms/step - loss: 0.0068 - accuracy: 0.9980 - val_loss: 0.0356 - val_ac

In [10]:
!mv /content/best_model.keras /content/drive/MyDrive/VIPERC/

In [11]:
from tensorflow.keras.models import load_model

best_model = load_model('/content/drive/MyDrive/VIPERC/best_model.keras')

In [13]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

test_loss, test_accuracy = best_model.evaluate(test_generator)
print(f'\nTest Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

class_labels = list(test_generator.class_indices.keys())
predictions = best_model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Lấy nhãn thực tế từ generator
true_classes = test_generator.classes

# Tính các chỉ số đánh giá
accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

# In ra kết quả
print('Accuracy:', accuracy * 100)
print('Precision:', precision * 100)
print('Recall:', recall * 100)
print('F1 Score:', f1* 100)

# In báo cáo chi tiết
print('\nClassification Report:')
print(classification_report(true_classes, predicted_classes, target_names=class_labels, digits=4))

# Tính accuracy cho từng nhãn
conf_matrix = confusion_matrix(true_classes, predicted_classes)
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

print('\nAccuracy for each class:')
for class_name, class_accuracy in zip(class_labels, class_accuracies):
    print(f'{class_name}: {class_accuracy:.2f}')

40/40 [==============================] - 5s 126ms/step - loss: 0.0220 - accuracy: 0.9936

Test Loss: 0.0220
Test Accuracy: 99.36%
40/40 [==============================] - 6s 145ms/step
Accuracy: 99.36305732484077
Precision: 99.36823397263723
Recall: 99.36305732484077
F1 Score: 99.36162828388979

Classification Report:
              precision    recall  f1-score   support

 cong_chieng     1.0000    1.0000    1.0000        60
     dan_bau     1.0000    1.0000    1.0000        79
      dan_co     1.0000    1.0000    1.0000        47
      dan_da     0.9918    0.9837    0.9878       123
     dan_day     1.0000    1.0000    1.0000        92
  dan_nguyet     0.9919    0.9760    0.9839       125
     dan_sen     0.9850    1.0000    0.9924       131
  dan_t_rung     1.0000    1.0000    1.0000       171
    dan_tinh     1.0000    1.0000    1.0000       124
   dan_tranh     0.9819    0.9939    0.9879       164
   dan_ty_ba     1.0000    0.9583    0.9787        48
        khen     1.0000    1.00